![Example grocery store](grocery_store.jpg)


In the bustling world of retail, especially in grocery stores, understanding consumer preferences and shopping habits is pivotal for business optimization. Imagine a local grocery chain seeking to enhance customer satisfaction and drive sales. By analyzing transactional data from two of its 24-hour stores, each with unique data storage practices due to the store owners' diverse backgrounds, the chain aims to uncover hidden patterns in consumer behavior. The goal here is multifaceted: optimizing store layouts, tailoring marketing strategies, and efficiently managing inventory to align with customer preferences. For instance, if analysis reveals that customers frequently purchase certain products together or show a preference for shopping at specific hours, the store can adjust its marketing efforts and stock levels accordingly. Additionally, understanding seasonal trends and weekly fluctuations in sales helps in planning promotions and staffing. Acting as the retail data scientist for this chain, you'll leverage detailed retail data to drive strategic decisions, ultimately enhancing customer experience and boosting the store's performance. 

`grocery_data1.csv` and `grocery_data2.csv` contain detailed records of grocery transactions from these two stores with the same column names:

| Variable         | Description                                           |
|------------------|-------------------------------------------------------|
| `CustomerID`     | Unique identifier for each customer                   |
| `DateRaw`        | Raw date of the transaction                           |
| `Time`           | Time of the transaction                               |
| `TransactionID`  | Unique identifier for each transaction                |
| `ProductName`    | Name of the product purchased                         |
| `PriceUSD`       | Price of the product in US dollars                    |
| `Quantity`       | Number of the product purchased                     |
| `PaymentMethod`  | Payment type used for the transaction               |
| `Category`       | Category of the product                               |


In [60]:
# Import packages
library(tidyverse)
library(dplyr)
library(lubridate)
library(readr)

# Start your code here
data_1 <- read_csv("grocery_data1.csv")
data_2 <- read_csv("grocery_data2.csv")

data <- bind_rows(data_1, data_2)

data <- data %>% mutate(Year = str_extract(DateRaw, "2023"),
					    Month = str_extract(DateRaw, "\\b(January|February|March|April|May|June|July|August|September|October|November|December)\\b"),
						DateRaw = str_replace(DateRaw, "2023",""),
						DateRaw = str_replace(DateRaw, "\\b(January|February|March|April|May|June|July|August|September|October|November|December)\\b",""),
					    DateRaw = str_replace(DateRaw, ",",""),
					    Day = DateRaw,
					    Date = as.Date(paste(Day, Month, Year, sep = " "), format = "%d %B %Y"))

data <- data %>% mutate(Sales = PriceUSD * Quantity,
					    WeekNum = week(Date))

Rows: 2619 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (4): DateRaw, ProductName, PaymentMethod, Category
dbl  (4): CustomerID, TransactionID, PriceUSD, Quantity
time (1): Time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 2581 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (4): DateRaw, ProductName, PaymentMethod, Category
dbl  (4): CustomerID, TransactionID, PriceUSD, Quantity
time (1): Time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [61]:
head(data)

CustomerID,DateRaw,Time,TransactionID,ProductName,PriceUSD,Quantity,PaymentMethod,Category,Year,Month,Day,Date,Sales,WeekNum
<dbl>,<chr>,<time>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<dbl>,<dbl>
41,28,20:00:00,2,Apples,5.64,5,Cash,Produce,2023,June,28,2023-06-28,28.20,26
170,18,06:00:00,3,Apples,17.92,1,Mobile Payment,Produce,2023,August,18,2023-08-18,17.92,33
86,18,09:00:00,4,Pasta,19.14,2,Mobile Payment,Grains,2023,August,18,2023-08-18,38.28,33
178,06,02:00:00,5,Rice,0.76,4,Debit Card,Grains,2023,August,06,2023-08-06,3.04,32
87,30,09:00:00,6,Chickpeas,11.30,1,Debit Card,Vegetarian,2023,July,30,2023-07-30,11.30,31
12,17,15:00:00,8,Onions,11.04,2,Debit Card,Produce,2023,June,17,2023-06-17,22.08,24


In [62]:
#1 Smallest Sales Deviation
tab1 <- data %>% group_by(WeekNum) %>% summarise(N=n(),Qts = sum(Quantity),TotalSales = sum(Sales))

tab1

WeekNum,N,Qts,TotalSales
<dbl>,<int>,<dbl>,<dbl>
22,135,411,3907.00
23,381,1168,12189.07
24,367,1121,11111.63
25,370,1110,11005.97
26,435,1351,13733.57
27,385,1179,11954.92
28,409,1196,12286.75
29,375,1158,12652.53
30,421,1241,12617.20


In [63]:
mean(tab1$TotalSales)

[1] 11442.88

In [64]:
tab1 <- tab1 %>% mutate(AbsoluteDeviation = abs(TotalSales - 11442.8814285714)) %>% arrange(AbsoluteDeviation)

In [65]:
tab1

WeekNum,N,Qts,TotalSales,AbsoluteDeviation
<dbl>,<int>,<dbl>,<dbl>,<dbl>
24,367,1121,11111.63,331.2514
25,370,1110,11005.97,436.9114
27,385,1179,11954.92,512.0386
34,403,1183,12144.57,701.6886
31,381,1136,12154.19,711.3086
23,381,1168,12189.07,746.1886
28,409,1196,12286.75,843.8686
30,421,1241,12617.20,1174.3186
29,375,1158,12652.53,1209.6486


In [66]:
smallest_sales_deviation <- 24

In [67]:
#2 Most Hourly Total Sales
tab2 <- data %>% mutate(Hour = str_sub(as.character(Time), 1,2))

tab2 <- tab2 %>% group_by(Hour) %>% summarise(N = n(), Qts = sum(Quantity), TotalSales = sum(Sales)) %>% arrange(desc(TotalSales))
tab2

Hour,N,Qts,TotalSales
<chr>,<int>,<dbl>,<dbl>
22,260,794,8476.95
08,265,818,8164.50
13,233,727,7523.40
06,236,742,7383.21
10,239,684,7286.96
05,227,703,7242.81
21,243,742,7235.86
00,224,666,7125.01
16,199,618,6897.30


In [68]:
most_hourly_sales <- 22

In [74]:
#3 Cornflake Days
tab3 <- data %>% filter(CustomerID == 107 & ProductName == "Cornflakes") %>%
				 arrange(Date) %>%
				 mutate(DaysDiff = c(0,diff(Date)))

tab3

CustomerID,DateRaw,Time,TransactionID,ProductName,PriceUSD,Quantity,PaymentMethod,Category,Year,Month,Day,Date,Sales,WeekNum,DaysDiff
<dbl>,<chr>,<time>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<dbl>,<dbl>,<dbl>
107,16,17:00:00,5015,Cornflakes,4.30,3,Debit Card,Cereal,2023,July,16,2023-07-16,12.90,29,0
107,22,05:00:00,3188,Cornflakes,18.24,1,Debit Card,Cereal,2023,July,22,2023-07-22,18.24,29,6
107,31,05:00:00,1056,Cornflakes,12.60,1,Credit Card,Cereal,2023,August,31,2023-08-31,12.60,35,40


In [70]:
diff(tab3$Date)

Time differences in days
[1]  6 40

In [71]:
cornflakes_days <- c(6,40)